In [ ]:
import cv2
import numpy as np
from collections import Counter
import heapq

# Function to build Huffman Tree
def build_huffman_tree(frequencies):
    heap = [[weight, [symbol, ""]] for symbol, weight in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)

        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]

        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])

    return sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[-1]), p))

# Function to encode image using Huffman codes
def huffman_encode_image(image, huffman_code):
    encoded_image = ''.join([huffman_code[pixel] for pixel in image.flatten()])
    return encoded_image

# Function to decode encoded image
def huffman_decode_image(encoded_data, huffman_code, shape):
    reverse_code = {v: k for k, v in huffman_code.items()}
    decoded_image = []

    code = ''
    for bit in encoded_data:
        code += bit
        if code in reverse_code:
            decoded_image.append(reverse_code[code])
            code = ''

    return np.array(decoded_image).reshape(shape)

# Example usage
image_path = 'image.png'  # Replace with your image path
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Step 1: Calculate frequency of each pixel value
freq = Counter(img.flatten())

# Step 2: Build Huffman Tree and codes
huffman_tree = build_huffman_tree(freq)
huffman_code = {item[0]: item[1] for item in huffman_tree}

# Step 3: Encode the image
encoded_image = huffman_encode_image(img, huffman_code)
print(f"Original image size: {img.size * 8} bits")
print(f"Encoded image size: {len(encoded_image)} bits")

# Step 4: Decode the image back
decoded_image = huffman_decode_image(encoded_image, huffman_code, img.shape)

# Show the original and decoded image
cv2.imshow("Original Image", img)
cv2.imshow("Decoded Image", decoded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
